In [27]:
from pathlib import Path
import pickle
import yaml
from collections import defaultdict

In [28]:
num_classes = 80
label_path = Path("D:/ml/code/yolov5/data/coco128.yaml")

In [29]:
val_dir = Path("D:/ml/code/datasets/coco128/labels/train2017")
pred_dir = Path("D:/ml/code/yolov5/runs/detect/coco128/yolov5s/labels")

In [30]:
assert label_path.exists(), f"{label_path} not exists"
assert val_dir.exists(), f"{val_dir} not exists"
assert pred_dir.exists(), f"{pred_dir} not exists"

# 创建id2txt的字典

```python
{
    0: ["pic1", "pic2", "pic3"...],
    1: ["pic2", "pic3", "pic5"...],
    ...
}
```

In [31]:
val_id2stem = defaultdict(list)
pred_id2stem = defaultdict(list)
val_id2stem, pred_id2stem

(defaultdict(list, {}), defaultdict(list, {}))

# 读取txt中的数据

In [32]:
def get_txts_data(path: Path) -> dict[str, list[str]]:
    """获取txt中数据,按照行分割

    Args:
        path (Path): txt文件夹路径

    Returns:
        dict[str, list[str]]:
            {
                img1: [line1],
                img2: [line1, line2],
                ....
            }
    """
    val_txts = path.glob("*.txt")
    data = {}
    for txt in val_txts:
        stem = txt.stem
        with open(txt) as f:
            lines = f.read().splitlines()                   # splitlines可以去除单个空行
            lines = [line for line in lines if line != ""]  # 去除空行
        data[stem] = lines
    return data

In [33]:
val_data = get_txts_data(val_dir)
print(len(val_data.keys()))
k = list(val_data.keys())[0]
print(k, ":", val_data[k])

128
000000000009 : ['45 0.479492 0.688771 0.955609 0.5955', '45 0.736516 0.247188 0.498875 0.476417', '50 0.637063 0.732938 0.494125 0.510583', '45 0.339438 0.418896 0.678875 0.7815', '49 0.646836 0.132552 0.118047 0.096937', '49 0.773148 0.129802 0.090734 0.097229', '49 0.668297 0.226906 0.131281 0.146896', '49 0.642859 0.079219 0.148063 0.148062']


In [34]:
pred_data = get_txts_data(pred_dir)
print(len(pred_data.keys()))
k = list(pred_data.keys())[0]
print(k, ":", pred_data[k])

126
000000000009 : ['60 0.486719 0.50625 0.973437 0.9875 0.27229', '50 0.589063 0.928125 0.15625 0.14375 0.336526', '45 0.642187 0.728125 0.503125 0.53125 0.338169', '50 0.647656 0.736458 0.495313 0.527083 0.507882', '45 0.75625 0.248958 0.453125 0.497917 0.776129']


# 获取 unique id

In [35]:
def get_txt_unique_ids(data: dict[str, list[str]]) -> dict[str, list[int]]:
    """获取每张图片中的类别id,去重

    Args:
        data (dict[str, list[str]]): get_txts_data 的返回值

    Returns:
        dict[str, list[int]]:
            {
                img1: [0],
                img2: [0, 1],
                ...
            }
    """
    unique_data = {}
    # key: filename
    # value: [line1, line2...]
    for stem, lines in data.items():
        # get line [0] label
        ids = [int(line.split(" ")[0]) for line in lines]
        # unique label
        unique_ids = sorted(set(ids))
        unique_data[stem] = unique_ids
    return unique_data

In [36]:
val_txt_unique_ids = get_txt_unique_ids(val_data)
print(len(val_txt_unique_ids.keys()))
k = list(val_txt_unique_ids.keys())[0]
print(k, ":", val_txt_unique_ids[k])

128
000000000009 : [45, 49, 50]


In [37]:
pred_txt_unique_ids = get_txt_unique_ids(pred_data)
print(len(pred_txt_unique_ids.keys()))
k = list(pred_txt_unique_ids.keys())[0]
print(k, ":", pred_txt_unique_ids[k])

126
000000000009 : [45, 50, 60]


# 将txt中的数据转移到id2stem中

In [38]:
val_id2stem[0], val_id2stem[1]

([], [])

In [39]:
def txt_id2stem(txt_unique_ids: dict[str, list[int]], id2stem: dict[int, list[str]]):
    """将uniqueid中的图片数据放入

    Args:
        txt_unique_ids (dict[str, list[int]]): get_txt_unique_ids 的返回值
        id2stem (dict[int, list[str]]):
            {
                0: [img1],
                1: [img2, img4],
                ...
            }
    """
    # 清空id2txt中数组的内容
    for i in id2stem.values():
        i.clear()
    # stem: filename
    # unique_ids: [id0, id1...]
    for stem, unique_ids in txt_unique_ids.items():
        for unique_id in unique_ids:
            # {id0: [img1, img2...]}
            id2stem[unique_id].append(stem)

In [40]:
txt_id2stem(val_txt_unique_ids, val_id2stem)
val_id2stem[1]

['000000000074', '000000000531', '000000000641']

In [41]:
txt_id2stem(pred_txt_unique_ids, pred_id2stem)
pred_id2stem[1]

['000000000074', '000000000086', '000000000641']

In [42]:
for i in range(num_classes):
    print(len(val_id2stem[i]), len(pred_id2stem[i]))

61 62
3 3
12 10
4 4
5 5
5 5
3 2
5 4
2 1
4 2
0 0
2 2
0 0
5 4
2 2
4 5
9 7
1 1
0 0
0 0
4 4
1 1
2 2
4 4
4 4
4 4
9 3
6 4
2 2
5 5
1 2
2 1
6 4
2 2
4 3
4 4
3 2
0 0
5 5
6 8
5 6
10 9
6 3
7 5
5 4
9 9
1 1
0 0
2 0
1 0
4 5
3 3
1 1
5 5
2 2
4 5
9 7
5 5
9 9
3 3
10 9
2 1
2 3
2 0
2 0
5 3
0 0
5 4
3 3
5 5
0 0
4 3
5 5
6 5
8 7
2 4
1 0
6 5
0 0
2 1


# 获取评估指标,每个类别和总的召回率,精确率

In [45]:
def get_metrics(val_id2stem: dict[int, list[str]], pred_id2stem: dict[int, list[str]], eps: float = 1e-8) -> tuple[dict, dict[int, dict]]:
    """获取每个类别和总的评估指标

    Args:
        val_id2stem (dict[int, list[str]]):  val
        pred_id2stem (dict[int, list[str]]): pred
        eps (float): 防止除零. Default 1e-8

    Returns:
        tuple[dict, dict[int, dict]]:
            dict:               总指标
            dict[int, dict]:    各类别指标
    """
    total_val = 0       # val total
    total_pred = 0      # pred total
    total_detect = 0    # detect total
    result_for_cls = {} # 每个类别准确率
    for i in range(num_classes):
        # set在这里不为去重,仅仅为了求交集,差集
        val_stem_for_single_id: set = set(val_id2stem[i])   # 单一类别val图片名字
        pred_stem_for_single_id: set = set(pred_id2stem[i]) # 单一类别pred图片名字

        num_val = len(val_stem_for_single_id)
        total_val += num_val
        num_pred = len(pred_stem_for_single_id)
        total_pred += num_pred

        # 检测到的图片名字
        detect      = val_stem_for_single_id.intersection(pred_stem_for_single_id)
        num_detect  = len(detect)
        total_detect += num_detect
        # 没有检测到图片名字
        not_detect  = val_stem_for_single_id.difference(pred_stem_for_single_id)
        # 过度检测到图片名字
        over_detect = pred_stem_for_single_id.difference(val_stem_for_single_id)

        recall = num_detect / (num_val + eps)
        precision = num_detect / (num_pred + eps)
        f1_score = (2 * recall * precision) / (recall + precision + eps)

        # 按照类别id保存
        result_for_cls[i] = {
            "num_val": num_val,
            "num_pred": num_pred,
            "num_detect": num_detect,
            "recall": recall,
            "precision": precision,
            "f1_score": f1_score,
            "detect": detect,
            "not_detect": not_detect,
            "over_detect": over_detect,
        }

    recall = total_detect / total_val
    precision = total_detect / total_pred
    f1_score = (2 * recall * precision) / (recall + precision+ eps)

    total_result = {
        "num_val": total_val,
        "num_pred": total_pred,
        "num_detect": total_detect,
        "recall": recall,
        "precision": precision,
        "f1_score": f1_score,
    }

    return total_result, result_for_cls

In [46]:
total_result, result_for_cls = get_metrics(val_id2stem, pred_id2stem)
total_result

{'num_val': 362,
 'num_pred': 323,
 'num_detect': 287,
 'recall': 0.7928176795580111,
 'precision': 0.8885448916408669,
 'f1_score': 0.8379557060006162}

In [47]:
result_for_cls

{0: {'num_val': 61,
  'num_pred': 62,
  'num_detect': 59,
  'recall': 0.9672131147540983,
  'precision': 0.9516129032258065,
  'f1_score': 0.9593490935292447,
  'detect': {'000000000036',
   '000000000049',
   '000000000061',
   '000000000074',
   '000000000077',
   '000000000086',
   '000000000109',
   '000000000110',
   '000000000113',
   '000000000136',
   '000000000149',
   '000000000151',
   '000000000165',
   '000000000192',
   '000000000201',
   '000000000241',
   '000000000257',
   '000000000260',
   '000000000294',
   '000000000308',
   '000000000315',
   '000000000322',
   '000000000326',
   '000000000328',
   '000000000338',
   '000000000357',
   '000000000360',
   '000000000368',
   '000000000370',
   '000000000382',
   '000000000389',
   '000000000395',
   '000000000397',
   '000000000415',
   '000000000419',
   '000000000428',
   '000000000431',
   '000000000436',
   '000000000443',
   '000000000446',
   '000000000459',
   '000000000474',
   '000000000488',
   '0000000005

# load label name

In [48]:
with open(label_path, 'r', encoding='utf-8') as f:
    dataset = yaml.safe_load(f)
names = dataset["names"]
names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [49]:
name2acc = {}
for name, acc_cls in zip(names.values(), result_for_cls.values()):
    name2acc[name] = {
        "num_val": acc_cls["num_val"],
        "num_pred": acc_cls["num_pred"],
        "num_detect": acc_cls["num_detect"],
        "recall": acc_cls["recall"],
        "precision": acc_cls["precision"],
        "f1_score": acc_cls["f1_score"],
    }
name2acc

{'person': {'num_val': 61,
  'num_pred': 62,
  'num_detect': 59,
  'recall': 0.9672131147540983,
  'precision': 0.9516129032258065,
  'f1_score': 0.9593490935292447},
 'bicycle': {'num_val': 3,
  'num_pred': 3,
  'num_detect': 2,
  'recall': 0.6666666666666666,
  'precision': 0.6666666666666666,
  'f1_score': 0.6666661666670417},
 'car': {'num_val': 12,
  'num_pred': 10,
  'num_detect': 9,
  'recall': 0.75,
  'precision': 0.9,
  'f1_score': 0.8181813223143503},
 'motorcycle': {'num_val': 4,
  'num_pred': 4,
  'num_detect': 3,
  'recall': 0.75,
  'precision': 0.75,
  'f1_score': 0.7499995000003333},
 'airplane': {'num_val': 5,
  'num_pred': 5,
  'num_detect': 5,
  'recall': 1.0,
  'precision': 1.0,
  'f1_score': 0.99999950000025},
 'bus': {'num_val': 5,
  'num_pred': 5,
  'num_detect': 4,
  'recall': 0.8,
  'precision': 0.8,
  'f1_score': 0.7999995000003126},
 'train': {'num_val': 3,
  'num_pred': 2,
  'num_detect': 1,
  'recall': 0.3333333333333333,
  'precision': 0.5,
  'f1_score': 0.

In [50]:
total_result

{'num_val': 362,
 'num_pred': 323,
 'num_detect': 287,
 'recall': 0.7928176795580111,
 'precision': 0.8885448916408669,
 'f1_score': 0.8379557060006162}

In [51]:
name2acc["total"] = total_result

# pandas

In [52]:
import pandas as pd

In [54]:
df = pd.DataFrame(name2acc, index=["num_val", "num_pred", "num_detect", "recall", "precision", "f1_score"]).T
df

,num_val,num_pred,num_detect,recall,precision,f1_score
person,61.0,62.0,59.0,0.967213,0.951613,0.959349
bicycle,3.0,3.0,2.0,0.666667,0.666667,0.666666
car,12.0,10.0,9.0,0.750000,0.900000,0.818181
motorcycle,4.0,4.0,3.0,0.750000,0.750000,0.750000
airplane,5.0,5.0,5.0,1.000000,1.000000,1.000000
...,...,...,...,...,...,...
scissors,1.0,0.1,0.0,0.000000,0.000000,0.000000
teddy bear,6.0,5.0,5.0,0.833333,1.000000,0.909090
hair drier,0.1,0.1,0.0,0.000000,0.000000,0.000000
toothbrush,2.0,1.0,1.0,0.500000,1.000000,0.666666


In [ ]:
# df.to_excel("result.xlsx")